In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
from numpy.polynomial import Polynomial
import sympy
from tqdm import tqdm
import helpers
import stopit
import sys
from scipy.stats import sem

# Constants
NUM_EXPERIMENT_ITERATIONS = 100
NUM_POLYNOMIALS_PER_EXPERIMENT = 10
MAX_VARS = 10
MAX_TERMS = 10
MAX_COEFF = 10
MAX_OFFSET = 1000
MAX_SYMBOLS_PER_TERM = 2

In [ ]:
def evaluate(
        max_vars,
        max_terms,
        max_symbols_per_term,
        max_coeff,
        max_offset,
        steps,
        num_polynomials,
        verbose=True):
    # Repeatedly generate random polynomials, evaluate each strategy and return the win counts of each strategy
    ct_experiments_where_gd_wins = 0
    ct_experiments_where_random_wins = 0
    tracker = tqdm if verbose else lambda x: x
    for i in tracker(range(num_polynomials)):
        poly, symbols = helpers.random_sos_polynomial(
            max_vars=max_vars,
            max_terms=max_terms,
            max_symbols_per_term=max_symbols_per_term,
            max_coeff=max_coeff,
            max_offset=max_offset)

        grad_result = helpers.gradient_descent(poly=poly, steps=steps)
        random_result = helpers.random_search(poly, steps=steps)
        if grad_result.best_objective_value < random_result.best_objective_value:
            ct_experiments_where_gd_wins += 1
        elif random_result.best_objective_value < grad_result.best_objective_value:
            ct_experiments_where_random_wins += 1
        else:
            # Skip iterations where there is a tie
            pass

    # Return the map from strategy name to win count
    return ct_experiments_where_gd_wins, ct_experiments_where_random_wins



gd_win_rates = []
for num_steps_for_optimizer_to_take in [5, 10, 50, 100]:
    for i in tqdm(range(NUM_EXPERIMENT_ITERATIONS)):
        ct_experiments_where_gd_wins, ct_experiments_where_random_wins = evaluate(
            max_vars=MAX_VARS,
            max_terms=MAX_TERMS,
            max_symbols_per_term=MAX_SYMBOLS_PER_TERM,
            max_coeff=MAX_COEFF,
            max_offset=MAX_OFFSET,
            steps=num_steps_for_optimizer_to_take,
            num_polynomials=NUM_POLYNOMIALS_PER_EXPERIMENT,
            verbose=False)

        gd_win_rate = (
            ct_experiments_where_gd_wins / (ct_experiments_where_gd_wins + ct_experiments_where_random_wins))
        gd_win_rates.append(gd_win_rate)

    print("Steps: {} Mean GD Win Rate: {} Standard Error GD Win Rate: {}".format(
        num_steps_for_optimizer_to_take,
        np.mean(gd_win_rates),
        sem(gd_win_rates)))
